# Cornerstone Project - Eric Rodrigues de Carvalho

This notebook is a series of codes that serves as one of the primary tools to execute my cornerstone project as an undergraduated student called "SUPER-RESOLUÇÃO DE IMAGENS EM
TOMOGRAFIA COMPUTADORIZADA DE BAIXA
DOSAGEM: COMPARAÇÃO DE MÉTODOS DE
APRENDIZADO PROFUNDO" (SUPER-RESOLUTION OF IMAGES IN
COMPUTED TOMOGRAPHY OF LOW
DOSAGE: COMPARISON OF METHODS OF
DEEP LEARNING).

The main goals of this notebook are:

1. Instruct how to install the required Python packages, download and prepare the Dataset and Models used in the work. (Section 0)
2. Convert files from HDF5 (the source file) to actual PNG images. (Section 1)
3. Convert sinogram representation to actual and human-readable CT-scanned images. (Section 1)
4. Convert SRCNN model to ONNX-based in order to run in chaiNNer application. (Section 2)
5. Instruct how to use chaiNNer to reproduce our results. (Section 3)


## Section 0 - Python and Data Preparation

##### Requirements 
This repository require the following packages

- h5py 
- numpy 
- matplotlib 
- scikit-learn
- scikit-image
- onnx
- torch
- lpips
- pyiqa
- seaborn

Install them (and their requirements) using pip, or: 

```bash
pip install -r requirements.txt
```

##### Data preparation

The data used in this work was downloaded from [Zenodo](https://zenodo.org/records/3384092).

Two files were used:

- observation_test.zip ([Link](https://zenodo.org/records/3384092/files/observation_validation.zip?download=1))
- ground_truth_test.zip ([Link](https://zenodo.org/records/3384092/files/ground_truth_test.zip?download=1))

1. Download the two mentioned files
2. Unzip them in the Dataset/ folder. The files should be in a path like this: ./Dataset/observation_test/xxxxx.hdf5 and Dataset/ground_truth_test/xxxxx.hdf5

Now, you can run the Section 1 below (it will generate ~7000 images, so it will take some time).

##### Model Preparation

Five pre-trained deep learning models were used in this work, along with [chaiNNer](https://chainner.app/download).

1. The first model is the SRCNN model which can be download directly from [this link](https://www.dropbox.com/s/rxluu1y8ptjm4rn/srcnn_x2.pth?dl=0). If the link fails, you can go to this [this repository](https://github.com/yjn870/SRCNN-pytorch) to download it. You have to choose the "Test; Model 9-5-5; Scale 2." model. 
2. The second model is the HAT model which can be download directly from [this link](https://drive.google.com/file/d/16xtMezHvckdWEuSiOxcO-dgOlsI0rEUg/view?usp=drive_link). If the link fails, you can go to this [this repository](https://github.com/chaiNNer-org/spandrel#model-architecture-support) to download it. You have to choose the "HAT | Models" link and download the "HAT-L_SRx2_ImageNet-pretrain.pth" file. 
3. The third model is the Real ESRGAN model which can be download directly from [this link](https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth). If the link fails, you can go to this [this repository](https://github.com/chaiNNer-org/spandrel#model-architecture-support) to download it. You have to choose the "Real-ESRGAN Compact (SRVGGNet) | Models" link and download the "RealESRGAN_x2plus.pth" file. 
4. The fourth model is the SwinIR model which can be download directly from [this link](https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth). If the link fails, you can go to this [this repository](https://github.com/chaiNNer-org/spandrel#model-architecture-support) to download it. You have to choose the "SwinIR | Models" link and download the "001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth" file. 
5. The fifth model is the DAT model which can be download directly from [this link](https://drive.google.com/file/d/1AYfLMnIqSlOJyOGabaRI48TEJh440fsN/view?usp=drive_link). If the link fails, you can go to this [this repository](https://github.com/chaiNNer-org/spandrel#model-architecture-support) to download it. You have to choose the "DAT | Models" link and download the "DAT_x2.pth" file. 

Download them and put in the Models/ folder!

You can now run Section 2 below.


## Section 1 - HDF to BMP and Sinogram to human-readable CT

For this cornerstone project, only ground_truth_test.zip and observation_test.zip are needed. After unzipping, set your "root_path" variable based on your own desired folders' structure

This script processes two datasets (ground truth and observation) from HDF5 files, typically used in medical imaging or computed tomography reconstruction scenarios. It converts these data into BMP image format and saves them into pre-defined directories.

Key steps in the code:

Directory Creation: The script ensures that the directories for storing the images (both ground truth and observation) exist. If they don't, it creates them using the create_dir function.

Ground Truth and Observation Processing: It reads HDF5 files containing arrays of image data. The ground truth data is directly saved as images after rotating them. The observation data, on the other hand, undergoes more complex processing: it performs a reconstruction using the iradon function (Filtered Back Projection) and crops the resulting image to a standard size before saving.

Error Handling: The code handles potential errors in reading files. If an error occurs while processing observation data, the corresponding ground truth image is deleted to maintain data consistency.

FBP Reconstruction: The observation data is reconstructed using the iradon function from the skimage.transform library. This method applies a transformation called Filtered Back Projection, commonly used in medical imaging to reconstruct cross-sectional images from projection data (as in CT scans).


In [ ]:
"""
This script processes medical imaging data stored in HDF5 format. Specifically, it reads the
ground truth and observation test datasets, processes them into images, and saves them in BMP format.

The script does the following:
1. Creates directories for saving ground truth and observation images if they don't exist.
2. Reads ground truth and observation HDF5 files.
3. For each file:
   - Ground truth images are rotated and saved directly.
   - Observation data is used to reconstruct images using the Filtered Back Projection (FBP) technique,
     crop the result, and save the reconstructed images.
4. If an error occurs in reading the observation file, the corresponding ground truth image is deleted to
   ensure data consistency.

Dependencies:
- h5py: For reading HDF5 files.
- numpy: For numerical operations, including image manipulation.
- matplotlib: For saving images in BMP format.
- skimage: For performing the Filtered Back Projection (FBP) reconstruction.

"""

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import iradon

# Path to the dataset folder containing HDF5 files
root_path = "./"
dataset_path = root_path+ 'Dataset/'

# List of folder names within the dataset containing various datasets (train, test, validation)
hdf5_folders = [
    'ground_truth_train',
    'ground_truth_test',
    'ground_truth_validation',
    'observation_train',
    'observation_test',
    'observation_validation',
]

# Function to create directories if they do not exist


def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


# Process ground truth test data by saving images in a specific directory
ground_truth_test_images_dir = os.path.join(
    dataset_path, 'ground_truth_test_images')
create_dir(ground_truth_test_images_dir)

# Process observation test data by saving images in a specific directory
observation_test_images_dir = os.path.join(
    dataset_path, 'observation_test_images')
create_dir(observation_test_images_dir)

# Get a sorted list of ground truth and observation HDF5 files
ground_truth_files = sorted(os.listdir(
    os.path.join(dataset_path, hdf5_folders[1])))
observation_files = sorted(os.listdir(
    os.path.join(dataset_path, hdf5_folders[4])))

# Initialize counters for naming the saved image files
ground_truth_counter = 0
observation_counter = 0

# Process both ground truth and observation HDF5 files
for gt_file_name, obs_file_name in zip(ground_truth_files, observation_files):
    gt_file_path = os.path.join(dataset_path, hdf5_folders[1], gt_file_name)
    obs_file_path = os.path.join(dataset_path, hdf5_folders[4], obs_file_name)

    # Try opening and processing the ground truth HDF5 file
    try:
        with h5py.File(gt_file_path, 'r') as gt_hdf5_file:
            for j in range(len(gt_hdf5_file['data'])):
                # Extract the ground truth image data
                gt_data = gt_hdf5_file['data'][j]

                # Save the ground truth image after rotating 90 degrees
                gt_image_name = f'ground_truth_test_{ground_truth_counter:04d}.bmp'
                plt.imsave(os.path.join(ground_truth_test_images_dir,
                           gt_image_name), arr=np.rot90(gt_data, k=1), cmap='gray')
                ground_truth_counter += 1

        # Try opening and processing the corresponding observation HDF5 file
        try:
            with h5py.File(obs_file_path, 'r') as obs_hdf5_file:
                for j in range(len(obs_hdf5_file['data'])):
                    # Extract observation data and transpose for FBP reconstruction
                    obs_data = obs_hdf5_file['data'][j]
                    obs_data = np.transpose(obs_data)

                    # Generate angles for FBP reconstruction
                    theta = np.linspace(0., 180., max(
                        obs_data.shape), endpoint=False)

                    # Perform FBP (Filtered Back Projection) reconstruction
                    reconstruction_fbp = iradon(
                        obs_data, theta=theta, filter_name='ramp')

                    # Crop the reconstructed image to a fixed size (362x362)
                    crop_height = min(362, reconstruction_fbp.shape[0])
                    crop_width = min(362, reconstruction_fbp.shape[1])
                    start_row = (
                        reconstruction_fbp.shape[0] - crop_height) // 2
                    start_col = (reconstruction_fbp.shape[1] - crop_width) // 2
                    reconstruction_fbp = reconstruction_fbp[start_row:start_row +
                                                            crop_height, start_col:start_col + crop_width]

                    # Save the observation image
                    obs_image_name = f'observation_test_{observation_counter:04d}.bmp'
                    plt.imsave(os.path.join(observation_test_images_dir,
                               obs_image_name), arr=reconstruction_fbp, cmap='gray')
                    observation_counter += 1

        except OSError as e:
            # Error handling for observation files
            print(f"Error opening {obs_file_path}: {e}")
            # Remove the corresponding ground truth image if observation fails
            os.remove(os.path.join(ground_truth_test_images_dir, gt_image_name))
            ground_truth_counter -= 1  # Adjust the counter to maintain consistency

    except OSError as e:
        # Error handling for ground truth files
        print(f"Error opening {gt_file_path}: {e}")
        # Remove the corresponding observation file if ground truth processing fails
        if os.path.exists(obs_file_path):
            os.remove(obs_file_path)

## Section 2 - SRCNN as an ONNX framework

This code defines a Super-Resolution Convolutional Neural Network (SRCNN) designed for single-channel (grayscale) images. The SRCNN model is a simple but effective deep learning model that performs image super-resolution by mapping low-resolution images to high-resolution counterparts through three convolutional layers. This script loads pre-trained weights for the SRCNN, sets the model to evaluation mode, and exports it to the ONNX format.

Key components of the code:

Model Definition: The SRCNN class defines a neural network with three convolutional layers. Each layer applies a different level of feature extraction and image refinement.

The first layer uses 64 filters to extract features.
The second layer reduces the feature space to 32 dimensions.
The third layer outputs the reconstructed high-resolution image.
ReLU activation is applied after the first two convolutional layers to introduce non-linearity.
Loading Pre-Trained Weights: The script loads pre-trained weights (from a .pth file) into the SRCNN model. These weights are assumed to have been trained on an external dataset.

Evaluation Mode: Once the model is loaded with weights, it's set to evaluation mode. This ensures that certain layers (like dropout or batch normalization, if they existed) behave properly during inference.

Export to ONNX: The model is exported to the ONNX format using the torch.onnx.export function, with a dummy input tensor that simulates a grayscale (single-channel) image


In [ ]:
"""
This script defines and exports a Super-Resolution Convolutional Neural Network (SRCNN) model to the ONNX format.
SRCNN is designed for single-channel (grayscale) image input, and this implementation is based on the architecture
outlined in the SRCNN paper for image super-resolution.

The script includes the following steps:
1. Define the SRCNN model architecture.
2. Load pre-trained weights (state_dict) into the model.
3. Set the model to evaluation mode.
4. Export the model to ONNX format using a dummy single-channel image input for format specification.

Dependencies:
- torch: For defining and loading the SRCNN model and its weights.
- torch.onnx: For exporting the PyTorch model to ONNX format.

"""


import torch
from torch import nn
import torch.onnx

# Define the SRCNN model class for single-channel input images



class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        """
        Initializes the SRCNN model.

        Args:
            num_channels (int): Number of input channels (default is 1 for grayscale images).
        """
        super(SRCNN, self).__init__()
        # First convolutional layer with 64 filters, kernel size 9x9, and padding to keep output size same as input
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=9 // 2)

        # Second convolutional layer with 32 filters, kernel size 5x5, and padding to maintain spatial dimensions
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=5 // 2)

        # Third convolutional layer that outputs the final single-channel (grayscale) image
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=5 // 2)

        # ReLU activation function applied after the first two convolutional layers
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        Defines the forward pass through the network.

        Args:
            x (torch.Tensor): Input image tensor of shape [batch_size, num_channels, height, width].

        Returns:
            torch.Tensor: The output high-resolution image tensor after passing through the network.
        """
        # Pass input through the first convolutional layer followed by ReLU activation
        x = self.relu(self.conv1(x))

        # Pass through the second convolutional layer followed by ReLU activation
        x = self.relu(self.conv2(x))

        # Pass through the third convolutional layer to generate the final output image
        x = self.conv3(x)
        return x


# Create an instance of the SRCNN model for single-channel (grayscale) input
model = SRCNN(num_channels=1)

# Load the pre-trained model weights from the .pth file
# Adjust the path to where the pre-trained weights are located
state_dict = torch.load('Models/srcnn_x2.pth')
model.load_state_dict(state_dict)  # Load the weights into the model

# Set the model to evaluation mode to ensure correct behavior during inference (e.g., disabling dropout if present)
model.eval()

# Export the model to ONNX format
torch.onnx.export(
    model,                      # The model to be exported
    # Dummy input of size [batch_size, num_channels, height, width] for grayscale images
    torch.randn(1, 1, 224, 224),
    "Models/srcnn_x2.onnx",    # Path where the ONNX model will be saved
    export_params=True,          # Whether to store the trained weights inside the ONNX model
    verbose=False,               # Whether to print detailed logs during export
    opset_version=10             # ONNX opset version to ensure compatibility
)

## Section 3 - chaiNNEr

First, we have to edit the chaiNNEr template file to run it. Edit the "edit_chaiNNer_file.py" file, settings the three following variables properly:

- dataset_folder
- images_folder
- models_folder

Define them using the full path to the Dataset/, Models/ and Images/ folder. Examples considering Linux and Windows were provided.

Run the "edit_chaiNNer_file.py" file to generate the chaiNNer file ("CT Super Resolution_edited.chn") with the correct folders.

Open [chaiNNer](ttps://chainner.app/download) (download and extract/install it if you haven't already). 

Load the "CT Super Resolution_edited.chn" file inside chaiNNer. Install dependencies if chaiNNer asks to do it. 

Since we edited the file outside chaiNNer, using the "edit_chaiNNer_file.py" script, it will raise a warning. You can ignore it.

If everything is ready, you can click in the green "play" button on top of chaiNNer. It will process ~3500 images using 5 deep learning methods, so it will take some time. 

Then, you can go to the Part 2 of this repository - Image Evaluation.